In [2]:
# Raw Streaming Debug (no UI) - Test local streaming endpoint 
import asyncio, math, json, time, contextlib, os, ssl
from pathlib import Path

try:
    import websockets  # type: ignore
except ImportError:
    raise RuntimeError("Install websockets: pip install websockets")

# Local server configuration (confirmed working)
HOST = '127.0.0.1:8000'
USE_SECURE = False  # Local server uses HTTP/WS
LANGUAGE = 'ar'  # 'ar' | 'en' | 'auto'
PCM_PATH = Path('/home/lumi/beautyai/voice_tests/input_test_questions/pcm/q10.pcm')
FRAME_MS = 20
FAST = True  # if False, will pace real-time
TAIL_SILENCE_MS = 800
AUTO_CLOSE_AFTER_FINAL_S = 15.0  # Increased timeout

print(f"Config -> host={HOST} secure={USE_SECURE} language={LANGUAGE} file={PCM_PATH}")
assert PCM_PATH.exists(), f"PCM file not found: {PCM_PATH}"

# First, verify streaming endpoint is available
import requests
try:
    stream_status = requests.get(f"http://{HOST}/api/v1/ws/streaming-voice/status", timeout=5)
    print(f"✅ Streaming voice status: {stream_status.status_code}")
    if stream_status.status_code == 200:
        status_data = stream_status.json()
        print(f"   Enabled: {status_data.get('enabled')}, Active sessions: {status_data.get('active_sessions')}")
    else:
        print(f"❌ Stream status error: {stream_status.text}")
        
except Exception as e:
    print(f"❌ Local server connection failed: {e}")

async def stream_debug():
    pcm_bytes = PCM_PATH.read_bytes()
    samples_per_frame = int(16000 * FRAME_MS / 1000)
    frame_bytes = samples_per_frame * 2
    total_frames = math.ceil(len(pcm_bytes)/frame_bytes)
    scheme = 'wss' if USE_SECURE else 'ws'
    url = f"{scheme}://{HOST}/api/v1/ws/streaming-voice?language={LANGUAGE}"
    print(f"\nConnecting to {url}")
    print(f"Audio: {total_frames} frames, {len(pcm_bytes)} bytes")

    events = []
    final_transcript = None
    tts_complete = False
    first_event_time = None
    start = time.time()

    async def sender(ws):
        print("📤 [sender] Starting to send audio frames...")
        cursor = 0
        frame_index = 0
        
        while cursor < len(pcm_bytes):
            chunk = pcm_bytes[cursor: cursor+frame_bytes]
            cursor += frame_bytes
            frame_index += 1
            
            try:
                await ws.send(chunk)
                if frame_index % 100 == 0:  # Progress every 100 frames
                    print(f"   Sent frame {frame_index}/{total_frames}")
            except Exception as e:
                print(f"❌ [sender] Send error at frame {frame_index}: {e}")
                return
                
            if not FAST:
                await asyncio.sleep(FRAME_MS/1000)
        
        # Send trailing silence
        print("🔇 [sender] Sending trailing silence...")
        silence_frames = max(1, int(TAIL_SILENCE_MS/FRAME_MS))
        silence_chunk = b"\x00\x00" * samples_per_frame
        
        for i in range(silence_frames):
            try:
                await ws.send(silence_chunk)
            except Exception as e:
                print(f"❌ [sender] Trailing silence send error: {e}")
                break
            if not FAST:
                await asyncio.sleep(FRAME_MS/1000)
                
        print("✅ [sender] Finished sending all audio data")

    async def receiver(ws, final_event):
        nonlocal final_transcript, tts_complete, first_event_time
        message_count = 0
        
        try:
            while True:
                try:
                    msg = await asyncio.wait_for(ws.recv(), timeout=2.0)
                    message_count += 1
                except asyncio.TimeoutError:
                    print("⏰ [receiver] No message received for 2 seconds, continuing...")
                    continue
                except Exception as e:
                    print(f"🔚 [receiver] Recv ended after {message_count} messages: {e}")
                    break
                
                now = time.time()
                if first_event_time is None:
                    first_event_time = now
                    
                try:
                    data = json.loads(msg)
                except json.JSONDecodeError:
                    print(f"📄 [receiver] Non-JSON message #{message_count}, length={len(msg)}")
                    continue
                    
                events.append(data)
                etype = data.get('type')
                content = data.get('text', data.get('content', ''))[:100]  # Truncate long content
                print(f"📨 [event #{message_count}] {etype}: {content}")
                
                if etype == 'final_transcript' and final_transcript is None:
                    final_transcript = data.get('text')
                    print(f"🎤 FINAL TRANSCRIPT: {final_transcript}")
                    
                if etype == 'tts_complete':
                    tts_complete = True
                    print("🔊 TTS COMPLETE")
                    
                if final_transcript and tts_complete:
                    print("✅ Both transcript and TTS complete!")
                    final_event.set()
                    break
                    
        except Exception as e:
            print(f"❌ [receiver] Error: {e}")

    try:
        async with websockets.connect(url, ping_interval=30) as ws:
            print("🔗 WebSocket connected successfully!")
            
            final_event = asyncio.Event()
            
            # Start both sender and receiver
            sender_task = asyncio.create_task(sender(ws))
            receiver_task = asyncio.create_task(receiver(ws, final_event))
            
            # Wait for sender to complete
            await sender_task
            print("📤 Sender finished, waiting for processing...")
            
            # Wait for final event or timeout
            try:
                await asyncio.wait_for(final_event.wait(), timeout=AUTO_CLOSE_AFTER_FINAL_S)
                print("✅ Stream completed successfully!")
            except asyncio.TimeoutError:
                print(f"⏰ Timeout waiting for completion ({AUTO_CLOSE_AFTER_FINAL_S}s)")
            
            # Cancel receiver task
            receiver_task.cancel()
            
            try:
                await receiver_task
            except asyncio.CancelledError:
                pass
                
    except Exception as e:
        print(f"❌ WebSocket connection error: {e}")

    end = time.time()
    print("\n=== STREAMING DEBUG SUMMARY ===")
    summary = {
        'endpoint': f"{scheme}://{HOST}",
        'file': str(PCM_PATH),
        'bytes': len(pcm_bytes),
        'frames': total_frames,
        'language': LANGUAGE,
        'final_transcript': final_transcript,
        'tts_complete': tts_complete,
        'events_total': len(events),
        'first_event_latency_ms': int((first_event_time-start)*1000) if first_event_time else None,
        'duration_s': round(end-start, 3),
    }
    print(json.dumps(summary, ensure_ascii=False, indent=2))
    
    return summary

# Run the stream debug
result = await stream_debug()
print(f"\n🏁 Test completed. Success: {result['final_transcript'] is not None}")

Config -> host=127.0.0.1:8000 secure=False language=ar file=/home/lumi/beautyai/voice_tests/input_test_questions/pcm/q10.pcm
✅ Streaming voice status: 200
   Enabled: True, Active sessions: 0

Connecting to ws://127.0.0.1:8000/api/v1/ws/streaming-voice?language=ar
Audio: 269 frames, 171870 bytes
🔗 WebSocket connected successfully!
📤 [sender] Starting to send audio frames...
   Sent frame 100/269
   Sent frame 200/269
🔇 [sender] Sending trailing silence...
✅ [sender] Finished sending all audio data
📨 [event #1] ready: 
📤 Sender finished, waiting for processing...


⏰ [receiver] No message received for 2 seconds, continuing...
📨 [event #2] decoder_started: 
📨 [event #3] ingest_mode: 
📨 [event #4] partial_transcript: ما هي الفائدة الرئيسية لعلاج الضوء النبدي المكثف؟
📨 [event #5] metrics_snapshot: 
📨 [event #6] perf_cycle: 
📨 [event #7] perf_cycle: 
📨 [event #8] endpoint_event: 
📨 [event #9] perf_cycle: 
📨 [event #10] perf_cycle: 
📨 [event #11] endpoint_event: 
📨 [event #12] final_transcript: ما هي الفائدة الرئيسية لعلاج الضوء النبدي المكثف؟
🎤 FINAL TRANSCRIPT: ما هي الفائدة الرئيسية لعلاج الضوء النبدي المكثف؟
📨 [event #13] perf_cycle: 
📨 [event #14] assistant_pipeline_start: 
📨 [event #15] tts_start: 
📨 [event #16] endpoint_event: 
📨 [event #17] heartbeat: 
📨 [event #18] endpoint_event: 
📨 [event #19] endpoint_event: 
📨 [event #20] endpoint_event: 
📨 [event #21] endpoint_event: 
📨 [event #22] endpoint_event: 
📨 [event #23] assistant_response: الفائدة الرئيسية لعلاج الضوء النبدي المكثف (IPL) هي تقليل البقع الداكنة والتصبغات على البشرة، وتحسين
📨 [eve

In [ ]:
# Test remote WSS endpoint that user confirmed working in browser
# This is just to verify the connection format is correct
import asyncio
import ssl
import json

try:
    import websockets  # type: ignore
except ImportError:
    raise RuntimeError("Install websockets: pip install websockets")

async def test_remote_wss_connection():
    """Test the remote WSS endpoint format that user confirmed works in browser"""
    url = "wss://api.gmai.sa/api/v1/ws/streaming-voice?language=ar"
    print(f"Testing remote WSS connection: {url}")
    
    # Create SSL context for self-signed certificate
    ssl_context = ssl.create_default_context()
    ssl_context.check_hostname = False
    ssl_context.verify_mode = ssl.CERT_NONE
    
    try:
        # Try to connect to verify the endpoint format
        async with websockets.connect(url, ssl=ssl_context, ping_interval=30) as ws:
            print("✅ Remote WSS connection successful!")
            print("🎧 Connection headers confirm WebSocket upgrade worked")
            
            # Send a small test to see if we get a response
            await asyncio.sleep(1)  # Give server a moment
            
            # Check if any initial messages
            try:
                msg = await asyncio.wait_for(ws.recv(), timeout=3.0)
                data = json.loads(msg)
                print(f"📨 Received initial message: {data}")
            except asyncio.TimeoutError:
                print("⏰ No initial message (normal for streaming endpoint)")
            except Exception as e:
                print(f"📄 Initial message error: {e}")
                
            print("🔚 Closing test connection")
            
    except Exception as e:
        print(f"❌ Remote WSS connection failed: {e}")
        print("   This might be due to network restrictions or server configuration")

# Test the remote endpoint format
await test_remote_wss_connection()